In [ ]:
import pandas as pd
from config import DOWNLOAD_REF_GENOME, RESULTS_FOLDER, DATA_FOLDER, NORMAL_R1, NORMAL_R2, TUMOR_R1, TUMOR_R2, REFERENCE_CSV_FILE

In [ ]:
# Download reference human genome
if DOWNLOAD_REF_GENOME:
    !wget -O "$DATA_FOLDER/hg38.fa.gz" http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
    !gunzip "$DATA_FOLDER/hg38.fa.gz"

# Quality control

In [2]:
# Perform quality checks using tools like FastQC and summarize quality metrics (e.g., sequence counts, 
# per-base quality, read duplication levels).

# Decompress if needed
!gunzip "$DATA_FOLDER"/*.fastq.gz

# Run FASTQC
# The summary of quality metrics is provided in the pdf report
!fastqc "$DATA_FOLDER/$NORMAL_R1" "$DATA_FOLDER/$NORMAL_R2" "$DATA_FOLDER/$TUMOR_R1" "$DATA_FOLDER/$TUMOR_R2" -o "$RESULTS_FOLDER"

zsh:1: no matches found: data/*.fastq.gz
null
null
null
null
Started analysis of PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 5% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 10% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 15% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 20% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 25% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 30% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 35% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 40% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 45% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 50% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 55% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 60% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_001.fastq
Approx 65% complete for PA221MH-lib09-P19-Norm_S1_L001_R1_00

# Alignment and mutation calling

In [10]:
# Align the samples to the human genome using tools like Bowtie2 or BWA.
# https://gatk.broadinstitute.org/hc/en-us/articles/360035535912-Data-pre-processing-for-variant-discovery

--2025-01-16 17:39:04--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: 'data/hg38.fa.gz'

data/hg38.fa.gz       2%[                    ]  26.23M  74.4KB/s    eta 51m 53s^C
gunzip: data/hg38.fa.gz: unexpected end of file
gunzip: data/hg38.fa.gz: uncompress failed


In [8]:
# Identify somatic mutations present in the cancer sample but absent in the normal tissue.
# https://gatk.broadinstitute.org/hc/en-us/articles/360035894731-Somatic-short-variant-discovery-SNVs-Indels

gunzip: error writing to output: No space left on device
gunzip: data/hg38.fa.gz: uncompress failed


In [5]:
# Use the normal tissue to calculate the median background mutation level.
# The background mutation level accounts for sequencing errors or biases
# that can mimic true mutations. Determine how many reads per million are
# required to confidently call a given mutation.